**Reto semana 3**

Carlos Andrés Franco Gaitan.




1. Liberías



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

2. Obtener los datos

In [ ]:
path = "/content/1. BD2_CarCrash_Int.xlsx"
datos = pd.read_excel(path, sheet_name = 0)
print(datos.head())

   ID      County            City  Weekday  Severity  ViolCat  ClearWeather  \
0   1   SAN DIEGO       SAN DIEGO        7         1        8             0   
1   2    HUMBOLDT  UNINCORPORATED        4         1        8             1   
2   3     VENTURA          OXNARD        2         1       12             1   
3   4  STANISLAUS  UNINCORPORATED        4         1        1             1   
4   5   MENDOCINO  UNINCORPORATED        5         1        1             1   

   Month CrashType  Highway  Daylight  
0      1         A        0         1  
1      1         A        1         0  
2      2         A        0         1  
3      1         A        0         0  
4      1         A        1         0  


3. Chequear integridad de los datos

In [ ]:
print((datos.isna()).sum()) #no hay na entonces no se hace nada mas.

ID              0
County          0
City            0
Weekday         0
Severity        0
ViolCat         0
ClearWeather    0
Month           0
CrashType       0
Highway         0
Daylight        0
dtype: int64


4. Filtrar por condado de San Francisco

In [ ]:
datos_sf = datos[datos["County"] == "SAN FRANCISCO"]
print(datos_sf.head())
print(datos_sf.tail())


        ID         County           City  Weekday  Severity  ViolCat  \
907    908  SAN FRANCISCO  SAN FRANCISCO        7         1        1   
1580  1581  SAN FRANCISCO  SAN FRANCISCO        2         0        9   
1581  1582  SAN FRANCISCO  SAN FRANCISCO        7         0        3   
1586  1587  SAN FRANCISCO  SAN FRANCISCO        7         0        3   
1587  1588  SAN FRANCISCO  SAN FRANCISCO        3         0        3   

      ClearWeather  Month CrashType  Highway  Daylight  
907              1      3         A        1         1  
1580             1      2         A        0         0  
1581             1      1         A        0         0  
1586             1      3         A        0         0  
1587             1      3         A        0         0  
            ID         County           City  Weekday  Severity  ViolCat  \
112598  112599  SAN FRANCISCO  SAN FRANCISCO        5         0       11   
112599  112600  SAN FRANCISCO  SAN FRANCISCO        5         0       11 

5. Se cargan las variables de interés

In [ ]:
variables = datos_sf[["Highway", "Weekday", "ViolCat", "ClearWeather", "Month", "CrashType", "Daylight"]]

entrada = variables[["Weekday", "ViolCat", "ClearWeather", "Month", "CrashType", "Daylight"]]

salida = np.array(variables[["Highway"]])

print(variables.head())
print(variables.tail())
print(variables[["CrashType"]].value_counts())


#Transformar las letras de Crashtype a numeros
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()


entrada["CrashType"] = le.fit_transform(entrada["CrashType"])

print(entrada.head())
print(entrada.tail())
print(entrada[["CrashType"]].value_counts())


      Highway  Weekday  ViolCat  ClearWeather  Month CrashType  Daylight
907         1        7        1             1      3         A         1
1580        0        2        9             1      2         A         0
1581        0        7        3             1      1         A         0
1586        0        7        3             1      3         A         0
1587        0        3        3             1      3         A         0
        Highway  Weekday  ViolCat  ClearWeather  Month CrashType  Daylight
112598        0        5       11             1      8         G         0
112599        0        5       11             1      3         G         1
112600        0        6       10             0      4         G         0
112601        0        4       10             1      4         G         0
112602        0        5       12             1      2         G         1
CrashType
C            653
D            483
G            400
B            268
E            143
A             88


<ipython-input-36-2ca4330a3f6b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entrada["CrashType"] = le.fit_transform(entrada["CrashType"])


6. Se implementa el modelo

In [ ]:
from sklearn.naive_bayes import GaussianNB

mnb = GaussianNB()
mnb.fit(entrada, salida)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

7. Obtener información

In [ ]:
ct = mnb.class_count_

print("El total de los choques es de\n", len(entrada))

print("Como estan separados los datos\n", ct, "siendo", ct[0],
      "los choques fuera de autopistas (Highway) y", ct[1],
      "los choques en autopistas")

porcentajes = ct/len(entrada)

print("Los datos porcentualmente estan divididos en\n", porcentajes[0].round(2),
      "fuera de autopistas y", porcentajes[1].round(2), "en autopistas.")


media = mnb.theta_
print("La media de los accidentes por categoría es\n", media)

std = np.sqrt(mnb.var_)
print("La desviación estándar de los accidentes por categoría es\n", std)


print("El límite superior de los valores es\n", media + std)
print("El límite inferior de los valores es: \n", media - std)

objetivo = [[5, 1, 1, 3, 4, 0]]

mnb.predict(objetivo)

El total de los choques es de
 2081
Como estan separados los datos
 [1580.  501.] siendo 1580.0 los choques fuera de autopistas (Highway) y 501.0 los choques en autopistas
Los datos porcentualmente estan divididos en
 0.76 fuera de autopistas y 0.24 en autopistas.
La media de los accidentes por categoría es
 [[3.88417722 7.45696203 0.86265823 4.35189873 3.21772152 0.66265823]
 [3.9261477  4.1497006  0.79840319 6.69461078 2.27744511 0.62075848]]
La desviación estándar de los accidentes por categoría es
 [[1.95062542 3.52136997 0.34420782 2.3433645  1.87371962 0.47280262]
 [2.01603723 2.16198902 0.40119266 3.44163641 1.05765448 0.48519831]]
El límite superior de los valores es
 [[ 5.83480263 10.978332    1.20686605  6.69526323  5.09144114  1.13546084]
 [ 5.94218494  6.31168962  1.19959585 10.13624719  3.33509959  1.10595679]]
El límite inferior de los valores es: 
 [[1.9335518  3.93559205 0.51845041 2.00853424 1.3440019  0.18985561]
 [1.91011047 1.98771158 0.39721054 3.25297437 1.2197906

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


array([0])

8. Evaluacion de desempeño

In [ ]:
from re import S

ydp = mnb.predict(entrada)

from sklearn.metrics import confusion_matrix

matconf = confusion_matrix(salida, ydp)
print(matconf)

vn = matconf[0,0]
fp = matconf[0,1]
fn = matconf[1,0]
vp = matconf[1,1]


exactitud = (vn + vp)/(vn + vp + fp + fn)
print("La exactitud del modelo es\n", ((exactitud.round(2))*100), "%")


te = (fp + fn)/(vn + vp + fp + fn)
print("La tasa de error del modelo es\n", ((te.round(2))*100), "%")

sensibilidad = vp/(vp+fn)
print("La sensibilidad del modelo es\n", ((sensibilidad.round(2))*100), "%")


especificidad = vn/(vn+fp)
print("La especificidad del modelo es\n",
      ((especificidad.round(2))*100), "%")

precision = vp/(vp+fp)
print("La precisión del modelo es\n", ((precision.round(2))*100), "%")


precision_n = vn/(vn+fn)
print("La precisión del modelo para los negativos es\n",
      ((precision_n.round(2))*100), "%")

[[1365  215]
 [ 224  277]]
La exactitud del modelo es
 79.0 %
La tasa de error del modelo es
 21.0 %
La sensibilidad del modelo es
 55.00000000000001 %
La especificidad del modelo es
 86.0 %
La precisión del modelo es
 56.00000000000001 %
La precisión del modelo para los negativos es
 86.0 %


9. **Análisis** (Aquí se hace tanto el análisis como la respuesta a las preguntas)

La base de datos al ser filtrada para mostrar solo al condado de San Francisco cuenta con 2081 datos de accidentes, de estos 2081 accidentes, cerca del 25% sucedieron en autopistas, mientras que el restante sucedieron por fuera de estas.

Para el presente análisis se asume que los dias de la semana fueron almacenados según la norma ISO 8601 en donde el día 1 de la semana es el lunes, y el 7 el domingo.

El límite superior para el día de la semana en los accidentes fuera de autopistas es de  5.83 lo que corresponde a el viernes en la tarde, mientras que para los accidentes en autopistas corresponde a 5.94 lo que tambien indica el viernes pero acercandose mucho mas a la media noche.

Por otro lado, los limites inferiores corresponden a 1.91 para autopistas (lunes cerca de media noche) y 1.93 fuera de autopistas (lunes cerca de media noche)

Para la fuerza de impacto "Violcat" se asume que el nivel 1 es el impacto mas leve y el nivel 24 el mas fuerte, en esta el límite superior en autopistas es ligeramente sobre 6, mientras que es cercano a 11 fuera de autopistas, por otro lado, el límite inferior es cercano a 2 en autopistas y cercano a 4 fuera de estas.

El límite superior para el tipo de accidente dentro de autopistas es de 3.33 lo que es mas cercano al tipo D, mientras que fuera de autopistas es cercano a 5, que son accidentes tipo F.

Para los límites inferiores, en autopistas se pueden ver valores de 1.2 (accidentes tipo B) y fuera de autopistas 1.3 (accidentes tipo B). lo que demuestra menor disparidad de tipos accidentes en los límites inferiores que en los superiores.

Ante la ausencia de explicaciones a cada letra de los tipos de accidente, no se puede hacer un análisis mas profundo en este ambito.

El presente modelo logra una exactitud muy cercana a 80%, significando que clasifica correctamente si un choque sucedió en una autopista o no basandose en las variables utilizadas.

Luego, se observa como se alcanza una especificidad cercana a 90%, significando así que el modelo es especialmente bueno para identificar los valores negativos, los cuales corresponden a los accidentes que no suceden en autopistas.

Las métricas de evaluación del modelo fueron calculadas gracias a una matriz de confusión que se puede observar arriba, en donde el 1365 de arriba a la izquierda corresponde a verdaderos negativos, los 215 de arriba a la derecha a falsos positivos, los 224 de abajo a la izquierda a falsos negativos y finalmente los 277 de abajo a la derecha a verdaderos positivos.

Este modelo además alcanzó una tasa de error levemente superior al 20%, lo que está en linea con la exactitud cercana al 80% expuesta anteriormente.

Se tiene una sensibilidad del 55% que es bastante baja, lo que significa que el modelo no es muy bueno para predecir los accidentes que sucedan en autopistas.

Finalmente, la precisión también cercana al 90% significa que de los accidentes que predice que sucedieron en autopistas, 90% de estos fueron predecidos correctamente.

De estas dos últimas métricas podemos concluir que al modelo le cuesta predecir que accidente sucedió en una autopista, pero casi nunca falla cuando marca un accidente sucedido en autopista, lo que demuestra que es bastante conservador.

El accidente que se pidió que se predijera, ha sido predecido como no ocurrido en autopista.


Conclusión: si bien según este modelo la mayoría de los accidentes no suceden en autopistas, los accidentes suceden accidentes suelen suceder en promedio en los mismos días de la semana, y los accidentes fuera de autopista son mucho mas severos que los ocurridos dentro de estas, el presente modelo tiene un nivel bajo de sensibilidad, lo que lleva a pensar que subestima la cantidad de accidentes que suceden en autopistas, lo que dependiendo de el uso que se le de al modelo puede ser un factor decisivo, sin embargo se lograron valores muy favorables para el resto de las métricas lo que dado otro tipo de propósitos, podría representar que este modelo puede ser gran herramienta.
